## Churn prediction

This notebook analyzes the customer data from a telecommunications company and, based on the features of the clients that unsubscribe from the previous month, it will try to predict which clients will unsubscribe this month.

It could be very useful to manage market campaigns to those clients with a high probability of unsubscribe.

### Extract, clean and transform the data

In [2]:
#Start by importing the needed packages

import numpy as np
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
